In [1]:
import pandas as pd
import re
import ast
import datetime

*Nouvel algorithme*

Conserver tous les codes utilisés par les membres du board (au moins deux fois), 
mais ne retenir ensuite que les articles n’utilisant que des codes compris dans ces catégories 
(en continuant de ne pas tenir compte des tous les codes commençant par A, B, C, Y, et  L6 à L9)

In [2]:
TODAY = datetime.datetime.now().strftime('%Y-%m-%d')

JOURNALS_DB_SOURCE = './sources/SIOE_ARTICLES_1998_2019_source.xlsx'
CODES_BOARD_SOURCE = './sources/JEL_BOARD_source.xlsx'

JOURNALS_DB_OUTPUT = './output/SIOE_journals_1998_2019_{}.xlsx'.format(TODAY)
CODES_BOARD_OUTPUT = './output/JEL_BOARD_{}.xlsx'.format(TODAY)

### Regular expressions

In [3]:
# keep titles with specific keywords
goods_titles_pattern = re.compile(r'\bcontracts?(?:ual)?\b|\binstitutions?(?:nal)?\b|\borgani[sz]ations?\b',
                                  flags=re.IGNORECASE)
goods_titles_regex = lambda x: True if goods_titles_pattern.search(x) else False

In [4]:
# remove non relevant JEL codes
bad_codes_pattern = re.compile(r'[A-CY]\d{2}|L[6-9]\d{1}')
good_code = lambda x: 'NO_BAD_CODE' if bad_codes_pattern.search(x) else 'YES'

### Read data

In [5]:
df_articles = pd.read_excel(JOURNALS_DB_SOURCE)
df_articles.head(2)

,id,review,year,accept,title,jel,keywords,abstract,authors,affiliations
0,4420,American Economic Review,2019,NaN,"(Mis)Allocation, Market Power, and Global Oil ...","['D24', 'F23', 'L13', 'L71', 'Q35']",Cost; Crude Oil; Oil; Production,We propose an approach to measuring the misall...,"Asker, John; Collard-Wexler, Allan; De Loecker...",Columbia U; Duke U; Katholieke Universiteit Le...
1,4405,American Economic Review,2019,NaN,A Macroeconomic Model of Price Swings in the H...,"['E44', 'G21', 'R31']",Credit; Finance; Financial Market; Housing; Ma...,This paper shows that a macro model with segme...,"Garriga, Carlos; Manuelli, Rodolfo; Peralta-Al...",Federal Reserve Bank of St Louis; Washington U...


In [6]:
df_codes = pd.read_excel(CODES_BOARD_SOURCE)
df_codes.head(2)

,code,accept,total,name
0,D23,NaN,143,Organizational Behavior; Transaction Costs; Pr...
1,D72,NaN,122,"Political Processes: Rent-seeking, Lobbying, E..."


### Filter codes

**First filter** : don't accept [A-CY]\d{2}|L[6-9]\d{1}

In [7]:
df_codes['accept'] = df_codes.code.apply(good_code)

In [8]:
len(df_codes[df_codes.accept == 'NO_BAD_CODE'])

65

**Second filter** : too rare JEL codes with exceptions

In [9]:
def no_less_than_except_list(row, no_less, except_list):
    return 'NO_TOO_RARE' if row.total < no_less and row.code not in except_list else row.accept

In [10]:
no_less = 2
except_list = ['K23', 'L31', 'L41']
df_codes['accept'] = df_codes.apply(lambda row: no_less_than_except_list(row, no_less, except_list), axis = 1)

In [11]:
len(df_codes[df_codes.accept == 'NO_TOO_RARE'])

92

**Total of accepted codes**

In [12]:
len(df_codes[df_codes.accept == 'YES'])

327

### Create "styled" codes file 

In [13]:
def highlight_cells_codes(row):
    if row.accept == 'NO_BAD_CODE':
        return ['background-color: #ffc2c2'] * len(row)
    elif row.accept == 'NO_TOO_RARE':
        return ['background-color: #f4b5ff'] * len(row)
    else:
        return [''] * len(row)

In [14]:
df_codes.style.apply(lambda row: highlight_cells_codes(row), axis=1).to_excel(CODES_BOARD_OUTPUT,index=False)

### Create dictionary with the codes

In [15]:
codes_dic = dict(zip(df_codes.code, df_codes.accept))

### Match articles

- Uniquement des codes du board 
- Ne pas tenir compte des tous les codes commençant par A, B, C, Y, et  L6 à L9

In [16]:
remove_sectors_codes = lambda codes: [c for c in ast.literal_eval(codes) if good_code(c) == 'YES']

In [17]:
df_articles['jel_clean'] = df_articles.jel.apply(remove_sectors_codes)

In [18]:
df_articles.head(2)

,id,review,year,accept,title,jel,keywords,abstract,authors,affiliations,jel_clean
0,4420,American Economic Review,2019,NaN,"(Mis)Allocation, Market Power, and Global Oil ...","['D24', 'F23', 'L13', 'L71', 'Q35']",Cost; Crude Oil; Oil; Production,We propose an approach to measuring the misall...,"Asker, John; Collard-Wexler, Allan; De Loecker...",Columbia U; Duke U; Katholieke Universiteit Le...,"[D24, F23, L13, Q35]"
1,4405,American Economic Review,2019,NaN,A Macroeconomic Model of Price Swings in the H...,"['E44', 'G21', 'R31']",Credit; Finance; Financial Market; Housing; Ma...,This paper shows that a macro model with segme...,"Garriga, Carlos; Manuelli, Rodolfo; Peralta-Al...",Federal Reserve Bank of St Louis; Washington U...,"[E44, G21, R31]"


In [19]:
def accept_article(row):
    # jel = ast.literal_eval(row.jel) # convert to list
    
    matches = [codes_dic[code] for code in row.jel_clean if code in codes_dic and codes_dic[code] == 'YES']
    
    if len(row.jel_clean) == len(matches):
        return 'YES'
    elif goods_titles_regex(row.title): # if matches by keyword in the title
        return 'YES_TITLE'
    else:
        return 'NO'

In [20]:
df_articles['accept'] = df_articles.apply(lambda row: accept_article(row), axis=1)

**Total of accepted articles**

In [21]:
len(df_articles[df_articles.accept.str.contains('YES')])

17604

In [22]:
# Title special
len(df_articles[df_articles.accept == 'YES_TITLE'])

143

**Total rejected articles**

In [23]:
len(df_articles[df_articles.accept == 'NO'])

4659

In [24]:
def highlight_cells_articles(row):
    if row.accept == 'NO':
        return ['background-color: #ffc2c2'] * len(row)
    elif row.accept == 'YES_TITLE':
        return ['background-color: #cfffd3'] * len(row)
    else:
        return [''] * len(row)

In [25]:
df_articles.style.apply(lambda row: highlight_cells_articles(row), axis=1).to_excel(JOURNALS_DB_OUTPUT,index=False)

In [ ]:
len(df_codes[df_codes.accept == 'NO_BAD_CODE'])

In [ ]:
len(df_codes)